In [19]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
data = pd.read_csv('./data/credit-a.csv',header = None)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653 entries, 0 to 652
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       653 non-null    int64  
 1   1       653 non-null    float64
 2   2       653 non-null    float64
 3   3       653 non-null    int64  
 4   4       653 non-null    int64  
 5   5       653 non-null    int64  
 6   6       653 non-null    int64  
 7   7       653 non-null    float64
 8   8       653 non-null    int64  
 9   9       653 non-null    int64  
 10  10      653 non-null    int64  
 11  11      653 non-null    int64  
 12  12      653 non-null    int64  
 13  13      653 non-null    int64  
 14  14      653 non-null    float64
 15  15      653 non-null    int64  
dtypes: float64(4), int64(12)
memory usage: 81.8 KB


In [22]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,30.83,0.000,0,0,9,0,1.25,0,0,1,1,0,202,0.0,-1
1,1,58.67,4.460,0,0,8,1,3.04,0,0,6,1,0,43,560.0,-1
2,1,24.50,0.500,0,0,8,1,1.50,0,1,0,1,0,280,824.0,-1
3,0,27.83,1.540,0,0,9,0,3.75,0,0,5,0,0,100,3.0,-1
4,0,20.17,5.625,0,0,9,0,1.71,0,1,0,1,2,120,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,0,21.08,10.085,1,1,11,1,1.25,1,1,0,1,0,260,0.0,1
649,1,22.67,0.750,0,0,0,0,2.00,1,0,2,0,0,200,394.0,1
650,1,25.25,13.500,1,1,13,7,2.00,1,0,1,0,0,200,1.0,1
651,0,17.92,0.205,0,0,12,0,0.04,1,1,0,1,0,280,750.0,1


In [23]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,30.83,0.000,0,0,9,0,1.25,0,0,1,1,0,202,0.0,-1
1,1,58.67,4.460,0,0,8,1,3.04,0,0,6,1,0,43,560.0,-1
2,1,24.50,0.500,0,0,8,1,1.50,0,1,0,1,0,280,824.0,-1
3,0,27.83,1.540,0,0,9,0,3.75,0,0,5,0,0,100,3.0,-1
4,0,20.17,5.625,0,0,9,0,1.71,0,1,0,1,2,120,0.0,-1


In [24]:
X = data.iloc[:,:-1]

In [25]:
Y = data.iloc[:, -1].replace(-1,0)

In [26]:
Y.unique()

array([0, 1], dtype=int64)

In [27]:
X = torch.from_numpy(X.values).type(torch.float32)

In [28]:
X.shape

torch.Size([653, 15])

In [29]:
Y = torch.from_numpy(Y.values.reshape(-1,1)).type(torch.float32)

In [30]:
Y.size()

torch.Size([653, 1])

In [56]:
from torch import nn

In [57]:
model = nn.Sequential(
            nn.Linear(15,1),
            nn.Sigmoid()
)

In [58]:
model

Sequential(
  (0): Linear(in_features=15, out_features=1, bias=True)
  (1): Sigmoid()
)

In [64]:
loss_fn = nn.BCELoss()

In [60]:
opt = torch.optim.Adam(model.parameters(),lr = 0.0001)

In [61]:
batches = 16
no_of_batch = 653//16

In [62]:
epoches = 1000

In [65]:
for epoch in range(epoches):
    for i in range(no_of_batch):
        start = i * batches
        end = start + batches
        x = X[start:end]
        y = Y[start:end]
        y_pred = model(x)
        loss = loss_fn(y_pred,y)
        opt.zero_grad()
        loss.backward()
        opt.step()

In [67]:
model.state_dict()    #sigmoid(w1 * x1 + w2 * x2 + ... + w15 * x15 +b)

OrderedDict([('0.weight',
              tensor([[-1.0264e-01, -3.0313e-03, -3.7170e-03,  2.3939e-01,  1.7698e-01,
                       -1.6965e-02,  1.9290e-01, -1.8440e-01,  1.7299e+00,  3.1290e-01,
                       -1.9433e-01, -2.5839e-02,  4.2590e-02,  1.1371e-03, -3.3750e-04]])),
             ('0.bias', tensor([0.0079]))])

model(X).data.numpy()

In [69]:
((model(X).data.numpy() > 0.5).astype('int' ) == Y.numpy()).mean()

0.8667687595712098